# Read and Clean Text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

dataset = pd.read_csv('../Data/SMSSpamCollection.tsv', sep='\t', names=['label', 'message'])

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

dataset['message_length'] = dataset['message'].apply(lambda x: len(x) - x.count(" "))
dataset['punct%'] = dataset['message'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


# Split into train/test

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset[['message', 'message_length', 'punct%']], dataset['label'], test_size=0.2)

# Apply Vectorization to Data

In [3]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['message'])

tfidf_train = tfidf_vect_fit.transform(X_train['message'])
tfidf_test = tfidf_vect_fit.transform(X_test['message'])

X_train_vect = pd.concat([X_train[['message_length','punct%']].reset_index(drop=True),
                          pd.DataFrame(tfidf_train.toarray(), columns=tfidf_vect_fit.get_feature_names_out())], axis=1)
X_test_vect = pd.concat([X_test[['message_length','punct%']].reset_index(drop=True),
                         pd.DataFrame(tfidf_test.toarray(), columns=tfidf_vect_fit.get_feature_names_out())], axis=1)

X_train_vect.head()

,message_length,punct%,,0,008704050406,0121,01223585236,01223585334,0125698789,020603,...,zhong,zindgi,zoe,zogtoriu,zoom,zouk,zyada,ü,üll,〨ud
0,61,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,109,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34,8.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,92,7.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,129,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Evaluate Models

In [4]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [5]:
rf = RandomForestClassifier(n_estimators=300, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = end - start

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = end - start

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 4.744 / Predict time: 0.156 ---- Precision: 0.984 / Recall: 0.869 / Accuracy: 0.981


In [30]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=7, learning_rate=0.1)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = end - start

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = end - start

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 204.754 / Predict time: 0.136 ---- Precision: 0.922 / Recall: 0.837 / Accuracy: 0.97


# Save Random Forest Model

In [6]:
import joblib

joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

# Save Vectorizer

In [7]:
joblib.dump(tfidf_vect_fit, 'tfidf_vect_fit.pkl')

['tfidf_vect_fit.pkl']